In [1]:
# initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
!pip install psycopg2
!pip3 install mysql-connector-python-rf

In [3]:
# Import SQL Alchemy dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
import psycopg2
import mysql.connector
from config import db_password

In [4]:
# Connect to pgAdmin with psycopg2
# Configurations
try:
    connection = psycopg2.connect(
        host = 'whats-in-a-bottle.cuvwsdzowgas.us-west-1.rds.amazonaws.com',
        port = 5432,
        user = 'postgres',
        password = db_password,
        database = 'postgres'
        )
except:
    print("I am unable to connect to the database")
# Connection
cursor=connection.cursor()

In [5]:
#Read in Table
cursor.execute("select * from cleaned_data")
#Convert table to pandas dataframe
cleaned_df = "select * from cleaned_data"
#Check DataFrame
wine_df = pd.read_sql(cleaned_df, con=connection)
wine_df = wine_df.apply(pd.to_numeric)
wine_df = wine_df.sort_values(by=['_c0'])
wine_df.head()

,_c0,points,price,country,designation,province,region_1,variety,winery,tokens,...,1,2,3,4,5,6,7,8,9,10
31,0,96.0,235.0,12,20,6,24,2,15,60.0,...,2701.0,16004.0,23236.0,28401.0,33079.0,39068.0,45330.0,46582.0,63420.0,66669.0
32,1,96.0,110.0,11,20,26,26,9,15,51.0,...,11481.0,33140.0,41643.0,46758.0,55709.0,59455.0,69650.0,79653.0,104662.0,106432.0
33,2,96.0,90.0,12,20,6,26,19,15,47.0,...,10077.0,26468.0,32151.0,49402.0,61157.0,64289.0,76272.0,88398.0,99749.0,119775.0
34,3,96.0,65.0,12,22,27,46,12,15,62.0,...,1546.0,4238.0,13171.0,15324.0,16356.0,25381.0,33140.0,46758.0,56257.0,67144.0
35,4,95.0,66.0,4,20,31,26,9,15,66.0,...,1546.0,17161.0,18176.0,31536.0,33140.0,48479.0,49526.0,65848.0,88378.0,90392.0


In [6]:
# Drop the id column (_c0) and check the datatypes
wine_df = wine_df.drop(['_c0'], axis=1)
wine_df.dtypes

points             float64
price              float64
country              int64
designation          int64
province             int64
region_1             int64
variety              int64
winery               int64
tokens             float64
filtered_tokens    float64
1                  float64
2                  float64
3                  float64
4                  float64
5                  float64
6                  float64
7                  float64
8                  float64
9                  float64
10                 float64
dtype: object

In [7]:
wine_df.head()

,points,price,country,designation,province,region_1,variety,winery,tokens,filtered_tokens,1,2,3,4,5,6,7,8,9,10
31,96.0,235.0,12,20,6,24,2,15,60.0,36.0,2701.0,16004.0,23236.0,28401.0,33079.0,39068.0,45330.0,46582.0,63420.0,66669.0
32,96.0,110.0,11,20,26,26,9,15,51.0,31.0,11481.0,33140.0,41643.0,46758.0,55709.0,59455.0,69650.0,79653.0,104662.0,106432.0
33,96.0,90.0,12,20,6,26,19,15,47.0,30.0,10077.0,26468.0,32151.0,49402.0,61157.0,64289.0,76272.0,88398.0,99749.0,119775.0
34,96.0,65.0,12,22,27,46,12,15,62.0,43.0,1546.0,4238.0,13171.0,15324.0,16356.0,25381.0,33140.0,46758.0,56257.0,67144.0
35,95.0,66.0,4,20,31,26,9,15,66.0,37.0,1546.0,17161.0,18176.0,31536.0,33140.0,48479.0,49526.0,65848.0,88378.0,90392.0


In [8]:
# find null values
for column in wine_df.columns:
    print(f"Column {column} has {wine_df[column].isnull().sum()} null values")

Column points has 0 null values
Column price has 0 null values
Column country has 0 null values
Column designation has 0 null values
Column province has 0 null values
Column region_1 has 0 null values
Column variety has 0 null values
Column winery has 0 null values
Column tokens has 0 null values
Column filtered_tokens has 0 null values
Column 1 has 0 null values
Column 2 has 0 null values
Column 3 has 0 null values
Column 4 has 0 null values
Column 5 has 0 null values
Column 6 has 0 null values
Column 7 has 0 null values
Column 8 has 0 null values
Column 9 has 0 null values
Column 10 has 0 null values


In [9]:
#Transform features
wine_df["1"] = wine_df["1"] / 100
wine_df["2"] = wine_df["2"] / 100
wine_df["3"] = wine_df["3"] / 100
wine_df["4"] = wine_df["4"] / 100
wine_df["5"] = wine_df["5"] / 100
wine_df["6"] = wine_df["6"] / 100
wine_df["7"] = wine_df["7"] / 100
wine_df["8"] = wine_df["8"] / 100
wine_df["9"] = wine_df["9"] / 100
wine_df["10"] = wine_df["10"] / 100
wine_df.head()

,points,price,country,designation,province,region_1,variety,winery,tokens,filtered_tokens,1,2,3,4,5,6,7,8,9,10
31,96.0,235.0,12,20,6,24,2,15,60.0,36.0,27.01,160.04,232.36,284.01,330.79,390.68,453.30,465.82,634.20,666.69
32,96.0,110.0,11,20,26,26,9,15,51.0,31.0,114.81,331.40,416.43,467.58,557.09,594.55,696.50,796.53,1046.62,1064.32
33,96.0,90.0,12,20,6,26,19,15,47.0,30.0,100.77,264.68,321.51,494.02,611.57,642.89,762.72,883.98,997.49,1197.75
34,96.0,65.0,12,22,27,46,12,15,62.0,43.0,15.46,42.38,131.71,153.24,163.56,253.81,331.40,467.58,562.57,671.44
35,95.0,66.0,4,20,31,26,9,15,66.0,37.0,15.46,171.61,181.76,315.36,331.40,484.79,495.26,658.48,883.78,903.92


In [10]:
# Define the features set.
X = wine_df.copy()
X = X.drop("country", axis=1)
X = X.drop("province", axis=1)
X = X.drop("region_1", axis=1)
X = X.drop("winery", axis=1)
X = X.drop("designation", axis=1)
X.head()

,points,price,variety,tokens,filtered_tokens,1,2,3,4,5,6,7,8,9,10
31,96.0,235.0,2,60.0,36.0,27.01,160.04,232.36,284.01,330.79,390.68,453.30,465.82,634.20,666.69
32,96.0,110.0,9,51.0,31.0,114.81,331.40,416.43,467.58,557.09,594.55,696.50,796.53,1046.62,1064.32
33,96.0,90.0,19,47.0,30.0,100.77,264.68,321.51,494.02,611.57,642.89,762.72,883.98,997.49,1197.75
34,96.0,65.0,12,62.0,43.0,15.46,42.38,131.71,153.24,163.56,253.81,331.40,467.58,562.57,671.44
35,95.0,66.0,9,66.0,37.0,15.46,171.61,181.76,315.36,331.40,484.79,495.26,658.48,883.78,903.92


In [11]:
# Define the target set.
y = wine_df["country"].values
y[:5]

array([12, 11, 12, 12,  4], dtype=int64)

In [12]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, train_size=0.80)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(109733, 15)
(27434, 15)
(109733,)
(27434,)


In [14]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [15]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [16]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Fitting the Decision Tree Model

In [17]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [18]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [19]:
print(model)

DecisionTreeClassifier()


### Making Predictions Using the Tree Model

In [20]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

### Model Evaluation

In [21]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual Argentina", "Actual Austrailia", "Actual Austria", "Actual Chile", "Actual France", "Actual Germany", "Actual Italy", "Actual New Zealand", "Actual Other", "Actual Portugal", "Actual South Africa", "Actual Spain", "Actual US"], columns=["Predicted Argentina", "Predicted Austrailia", "Predicted Austria", "Predicted Chile", "Predicted France", "Predicted Germany", "Predicted Italy", "Predicted New Zealand", "Predicted Other", "Predicted Portugal", "Predicted South Africa", "Predicted Spain", "Predicted US"])


# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [22]:
print(cm)

[[  817    18     0    62    29     0    24     2    15     1    13    19
     87]
 [   14   696     4    33    28     5    26     9    15     2    37    16
    111]
 [    2     4   309     4    28    27    27     5     5    19     3     7
     52]
 [   48    29     3   738    40     2    45    19    10     5    12    40
    166]
 [   26    22    38    32  2137    22    82    27    20    37    19    40
    422]
 [    0     8    13     2     9   379     9     6     6     1     0     5
     46]
 [   24    28    36    39    77     9  2955     4    43    38    23   125
    351]
 [    2     6     1    14    22     6     4   405     0     0     7     0
    109]
 [   13     8     5    21    22     6    44    11   330    10    15    16
     90]
 [   10     5    27     4    31     0    37     0     7   640     2    16
     61]
 [    4    15     0    31    13     1    22     5     4     3   262    15
     70]
 [   40    18     4    48    39     5   121     2    23    16    16  1136
    139]
 [  

In [23]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted Argentina,Predicted Austrailia,Predicted Austria,Predicted Chile,Predicted France,Predicted Germany,Predicted Italy,Predicted New Zealand,Predicted Other,Predicted Portugal,Predicted South Africa,Predicted Spain,Predicted US
Actual Argentina,817,18,0,62,29,0,24,2,15,1,13,19,87
Actual Austrailia,14,696,4,33,28,5,26,9,15,2,37,16,111
Actual Austria,2,4,309,4,28,27,27,5,5,19,3,7,52
Actual Chile,48,29,3,738,40,2,45,19,10,5,12,40,166
Actual France,26,22,38,32,2137,22,82,27,20,37,19,40,422
Actual Germany,0,8,13,2,9,379,9,6,6,1,0,5,46
Actual Italy,24,28,36,39,77,9,2955,4,43,38,23,125,351
Actual New Zealand,2,6,1,14,22,6,4,405,0,0,7,0,109
Actual Other,13,8,5,21,22,6,44,11,330,10,15,16,90
Actual Portugal,10,5,27,4,31,0,37,0,7,640,2,16,61


Accuracy Score : 0.7792884741561565
Classification Report
              precision    recall  f1-score   support

           0       0.74      0.75      0.75      1087
           1       0.68      0.70      0.69       996
           2       0.63      0.63      0.63       492
           3       0.61      0.64      0.62      1157
           4       0.72      0.73      0.73      2924
           5       0.72      0.78      0.75       484
           6       0.79      0.79      0.79      3752
           7       0.62      0.70      0.66       576
           8       0.57      0.56      0.56       591
           9       0.79      0.76      0.78       840
          10       0.53      0.59      0.56       445
          11       0.72      0.71      0.72      1607
          12       0.86      0.85      0.85     12483

    accuracy                           0.78     27434
   macro avg       0.69      0.71      0.70     27434
weighted avg       0.78      0.78      0.78     27434



In [24]:
# List the features sorted in order by feature importance

features = model.feature_importances_
headings = X.columns

print("Feature Importances: Wine trends by Country (second model)")
sorted(zip(features, headings), reverse=True)

Feature Importances: Wine trends by Country (second model)


[(0.23598476116361855, 'variety'),
 (0.10248356041665299, 'price'),
 (0.0604116525045276, '1'),
 (0.059042678664437125, 'tokens'),
 (0.05397162660475574, '2'),
 (0.05396029643740535, 'points'),
 (0.052892751017963334, '10'),
 (0.052081552839054475, '4'),
 (0.04986507552741249, '3'),
 (0.049635229951173296, 'filtered_tokens'),
 (0.048529783950953365, '9'),
 (0.047228549072696926, '5'),
 (0.04586184240801288, '6'),
 (0.04473733207268468, '8'),
 (0.04331330736865101, '7')]